### Recommender Systems

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('movielens_ratings.csv')

In [3]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0


In [4]:
df.corr()

,movieId,rating,userId
movieId,1.000000,0.036569,0.003267
rating,0.036569,1.000000,0.056411
userId,0.003267,0.056411,1.000000


In [5]:
import numpy as np
df['mealskew'] = df['movieId'].apply(lambda id: np.nan if id > 31 else id)

In [6]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0
mealskew,486.0,15.502058,9.250634,0.0,7.0,15.0,23.0,31.0


In [7]:
mealmap = { 2. : "Chicken Curry",   
           3. : "Spicy Chicken Nuggest",   
           5. : "Hamburger",   
           9. : "Taco Surprise",  
           11. : "Meatloaf",  
           12. : "Ceaser Salad",  
           15. : "BBQ Ribs",  
           17. : "Sushi Plate",  
           19. : "Cheesesteak Sandwhich",  
           21. : "Lasagna",  
           23. : "Orange Chicken",
           26. : "Spicy Beef Plate",  
           27. : "Salmon with Mashed Potatoes",  
           28. : "Penne Tomatoe Pasta",  
           29. : "Pork Sliders",  
           30. : "Vietnamese Sandwich",  
           31. : "Chicken Wrap",  
           np.nan: "Cowboy Burger",   
           4. : "Pretzels and Cheese Plate",   
           6. : "Spicy Pork Sliders",  
           13. : "Mandarin Chicken PLate",  
           14. : "Kung Pao Chicken",
           16. : "Fried Rice Plate",  
           8. : "Chicken Chow Mein",  
           10. : "Roasted Eggplant ",  
           18. : "Pepperoni Pizza",  
           22. : "Pulled Pork Plate",   
           0. : "Cheese Pizza",   
           1. : "Burrito",   
           7. : "Nachos",  
           24. : "Chili",  
           20. : "Southwest Salad",  
           25.: "Roast Beef Sandwich"}

In [8]:
df['meal_name'] = df['mealskew'].map(mealmap)

In [9]:
df.to_csv('Meal_Info.csv',index=False)

In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName('recconsulting').getOrCreate()

In [12]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [13]:
data = spark.read.csv('Meal_Info.csv',inferSchema=True,header=True)

In [14]:
(training, test) = data.randomSplit([0.8, 0.2])

In [16]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

In [17]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

predictions.show()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+-------+------+------+--------+-------------+----------+
|movieId|rating|userId|mealskew|    meal_name|prediction|
+-------+------+------+--------+-------------+----------+
|     31|   4.0|    12|    31.0| Chicken Wrap|  1.543386|
|     31|   1.0|    13|    31.0| Chicken Wrap| 1.4337338|
|     31|   3.0|     8|    31.0| Chicken Wrap| 3.0498347|
|     31|   2.0|    25|    31.0| Chicken Wrap|-0.8862498|
|     31|   3.0|    14|    31.0| Chicken Wrap| 1.1693456|
|     31|   1.0|    18|    31.0| Chicken Wrap| 2.2382712|
|     85|   1.0|    26|    null|Cowboy Burger| 5.5169034|
|     85|   1.0|    12|    null|Cowboy Burger| 2.0326931|
|     85|   2.0|    20|    null|Cowboy Burger|-3.8783782|
|     85|   5.0|     8|    null|Cowboy Burger|0.51454866|
|     65|   1.0|    22|    null|Cowboy Burger| 4.1334677|
|     65|   2.0|     3|    null|Cowboy Burger|-2.6421788|
|     65|   1.0|    19|    null|Cowboy Burger| 1.7492521|
|     53|   3.0|    13|    null|Cowboy Burger| 0.5710138|
|     53|   1.